In [1]:
import tensorflow as tf
import cv2
import glob
import os
import numpy as np

In [2]:
train_path = "Dataset_augmented/train/"
test_path = "Dataset_augmented/test/"

test_img_path = glob.glob(test_path + "*")
train_img_path = glob.glob(train_path + "*")

In [3]:
def min_max(img):
    image = img.astype('float32')
    image = np.divide(image, 255.0)
    
    return image

In [4]:
# Load Test Image and Extract label and normalization
x_test = []
y_test = []
for idx, img_path in enumerate(test_img_path):
    temp = cv2.imread(img_path)
    temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
    
    # Label
    label = img_path.split("_")[4].split(".")[0]
    
#     x_test.append(min_max(temp))
    x_test.append(temp)
    y_test.append(label)

In [5]:
x_test = np.array(x_test)
y_test = np.array(y_test)

In [6]:
# Load Test Image and Extract label and normalization
x_train = []
y_train = []
for idx, img_path in enumerate(train_img_path):
    temp = cv2.imread(img_path)
    temp = cv2.cvtColor(temp, cv2.COLOR_BGR2RGB)
    
    # Label
    label = img_path.split("_")[4].split(".")[0]
    
    x_train.append(temp)
    y_train.append(label)

In [7]:
x_train = np.array(x_train)

In [8]:
y_train = np.array(y_train)

In [9]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=17)
y_test = to_categorical(y_test, num_classes=17)

In [10]:
def cnn_model():
    return tf.keras.Sequential([
        tf.keras.layers.Input(shape=(256, 256, 3)),
        tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same'), # 128
        tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=(2, 2), padding='same'), # 64
        tf.keras.layers.Activation("relu"),
        tf.keras.layers.Conv2D(128, kernel_size=(1, 1), strides=(1, 1), padding='same'), # 64
        tf.keras.layers.Conv2D(128, kernel_size=(1, 1), strides=(2, 2), padding='same'), # 64
        tf.keras.layers.Activation("relu"),
        tf.keras.layers.Conv2D(128, kernel_size=(1, 1), strides=(1, 1), padding='same'), # 32
        tf.keras.layers.Conv2D(128, kernel_size=(1, 1), strides=(1, 1), padding='same'), # 32
        tf.keras.layers.Activation("relu"),

        tf.keras.layers.Dropout(0.5), # 0.5 - 0.8

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, kernel_initializer='glorot_normal'),
        tf.keras.layers.Dense(128, kernel_initializer='glorot_normal'),
        tf.keras.layers.Activation("relu"),
        tf.keras.layers.Dense(128, kernel_initializer='glorot_normal'),
        tf.keras.layers.Dense(128, kernel_initializer='glorot_normal'),
        tf.keras.layers.Activation("relu"),

        tf.keras.layers.Dense(14, activation="softmax")
    ])

def create_resnetv250():
    return tf.keras.Sequential([
        tf.keras.applications.ResNet50V2(
            include_top=True,
            weights=None,
            input_shape=(224, 224, 3),
            classes=14,
            classifier_activation="softmax"
        )
    ])

In [11]:
import gc
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import Callback

class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()

In [12]:
model = cnn_model()

adam_optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    loss='categorical_crossentropy',
    optimizer=adam_optimizer,
    metrics=['accuracy']
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 128)     3584      
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 128)       147584    
                                                                 
 activation (Activation)     (None, 64, 64, 128)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 128)       16512     
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 128)       16512     
                                                                 
 activation_1 (Activation)   (None, 32, 32, 128)       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 32, 32, 128)       1

In [13]:
history = model.fit(
    x_train,
    y_train,
    steps_per_epoch=16,
    validation_steps=16,
    epochs=4,
    validation_split=0.1,
    validation_data=(x_test, y_test)
)

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [ ]:
# Save Train result
fig2 = plt.gcf()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

fig2.savefig('history/loss01.jpg')

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

fig1.savefig('history/accuracy01.jpg')

In [ ]:
# Save Weight
model.save('history/model01.h5')